In [1]:
import numpy as np
import pandas as pd

## Rule Based Item Recommendation Model

1. 필요한 데이터 로딩

In [156]:
interaction_matrix = pd.read_csv("/opt/ml/input/data/itemInteractionMatrix.csv")
item_feature = pd.read_excel("/opt/ml/workspace/crawler/item_crawler_ver2/total_asset/item.xlsx")
item_feature = item_feature[["id","name","url", "likes", "big_class","mid_class"]]
item_id_list = list(interaction_matrix.columns[1:])

2. 데이터 대분류 전처리

In [157]:
upper = ["상의", "기타 상의"]
lower = ["바지", "하의"]
outer = ["아우터"]
shoes = ["스니커즈", "신발", "스포츠신발"]
accessory = ["선글라스/안경테", "액세서리", "주얼리", "시계", "가방", "여성 가방", "모자", "스포츠모자", "스포츠가방"]
etc = ["양말/레그웨어", "디지털/테크", "뷰티", "속옷", "디지털/테크"]

In [203]:
actual_big_class_list = list()
for i in range(len(item_feature)) :
    item_info = item_feature.iloc[i]
    id = item_info["id"]
    big = item_info["big_class"]
    mid = item_info["mid_class"]
    if big in upper or mid in upper :
        actual_big_class = "상의"
    elif big in lower or mid in lower :
        actual_big_class = "하의"
    elif big in outer or mid in outer :
        actual_big_class = "아우터"
    elif big in shoes or mid in shoes :
        actual_big_class = "신발"
    elif big in accessory or mid in accessory :
        actual_big_class = "액세서리"
    else : 
        actual_big_class = "기타"
    actual_big_class_list.append(actual_big_class)

item_feature["my_big_class"] = actual_big_class_list

In [204]:
item_feature["likes"] = item_feature["likes"].fillna(0)

3. 모델 실행

In [205]:
item_id = 1432188
data = interaction_matrix[interaction_matrix["id"]==item_id]
data = data.to_numpy()[0][1:] 

In [206]:
index_list = list()
for i in range(len(data)) :
    if data[i] != 0 :
        index_list.append(i)

In [207]:
rec_result = {"상의" : [], "하의" : [], "아우터" : [], "신발" : [], "액세서리" : [], "기타" : []}
for index in index_list :
    rec_item_id = item_id_list[index]
    rec_item_info = item_feature[item_feature["id"]==int(rec_item_id)]

    big_class = rec_item_info["my_big_class"].iloc[0]
    name = rec_item_info["name"].iloc[0]
    url = rec_item_info["url"].iloc[0]
    
    rec_result[big_class].append((rec_item_id, name, url))

In [208]:
def get_best_item(item_list : list) -> list :
    likes_list = list()
    for item in item_list : 
        item_id = int(item[0])
        item_likes = item_feature[item_feature["id"]==item_id]["likes"].iloc[0]
        likes_list.append(item_likes)
    for i in range(len(likes_list)) :
        if likes_list[i] == max(likes_list) :
            return item_list[i]

In [209]:
for key in rec_result.keys() :
    
    if len(rec_result[key]) == 0:
        print(f"{key} 추천 아이템이 없습니다.")
    else :
        rec_item = get_best_item(rec_result[key])
        print(f"{key} 추천 아이템 : {rec_item[1]}, 상품 URL : {rec_item[-1]}")

상의 추천 아이템 : 콘트라스트 스티치 후디드 스웨트셔츠 [미디엄 그레이], 상품 URL : https://www.musinsa.com/app/goods/1817968/0
하의 추천 아이템 : 코튼 와이드 벌룬 팬츠 [크림], 상품 URL : https://www.musinsa.com/app/goods/2069532/0
아우터 추천 아이템 : 램스울 브이 넥 니트 베스트 [미디엄 그레이], 상품 URL : https://www.musinsa.com/app/goods/2100453/0
신발 추천 아이템 : RONDA CS2005 ALL BLACK, 상품 URL : https://www.musinsa.com/app/goods/1923386/0
액세서리 추천 아이템 : CARSON HQ 가죽 메신저 크로스 백, 상품 URL : https://www.musinsa.com/app/goods/2193016/0
기타 추천 아이템이 없습니다.
